<a href="https://colab.research.google.com/github/adssoccer1/twitterMiddleEastSentimentAnalyis/blob/master/copy_of_MiddleEast_Twitter_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tweepy  
!pip install tweet-preprocessor
!pip install googletrans
!python -m nltk.downloader stopwords
!python -m nltk.downloader punkt



In [0]:
#installing necessary packages

from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
import pandas as pd
import csv
import re #regular expression
from textblob import TextBlob
import string
import os



import preprocessor as p
import nltk
from nltk.corpus import stopwords

#nltk.download('stopwords')
#nltk.download('punkt')
#stopwords.download('stopwords')

In [0]:
#Twitter credentials for the app
consumer_key = ""
consumer_secret = ""
access_key= ""
access_secret = ""



In [0]:
import tweepy as tweepy
#pass twitter credentials to tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

In [0]:
COLS = ['id', 'created_at', 'source', 'original_text','clean_text', 'sentiment','polarity','subjectivity', 'lang',
'favorite_count', 'retweet_count', 'original_author',   'possibly_sensitive', 'hashtags',
'user_mentions', 'place', 'place_coord_boundaries']

In [0]:
#yyyy-mm-dd
start_date = '2018-05-05'
end_date = '2020-02-21'

In [0]:
#file location changed to "data/telemedicine_data_extraction/" for clearer path
kuwait_tweets = "/content/sample_data/OMAN2_GREEN.csv"

In [0]:
#HappyEmoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

In [0]:
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

In [0]:
#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)

In [0]:
#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

In [0]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def clean_tweets(tweet):
 
    stop_words = set(stopwords.words('arabic')) #    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(tweet)
#after tweepy preprocessing the colon symbol left remain after      #removing mentions
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
#replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
#remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)
#filter using NLTK library append it to a string
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []
#looping through conditions
    for w in word_tokens:
#check tokens against stop words , emoticons and punctuations
        if w not in stop_words and w not in emoticons and w not in string.punctuation:
            filtered_tweet.append(w)
    return ' '.join(filtered_tweet)
    #print(word_tokens)
    #print(filtered_sentence)return tweet

In [0]:
#Ar
"""http://docs.tweepy.org/en/v3.5.0/api.html Use this tweepy documentation to specify the
search. The search is interfaced with the tweepy.cursor(). can mess around with langs 
and keywords now....
"""
import googletrans
from googletrans import Translator
import copy
#translator = Translator()
import time

#set to check for duplicated tweets
setOfTweets = set()

#method write_tweets()
def write_tweets(keyword, file, numTweets):
    # If the file exists, then read the existing data from the CSV file.
    #if os.path.exists(file):
    #     df = pd.read_csv(file, header=0)
    #else:
    df = pd.DataFrame(columns=COLS)
    #page attribute in tweepy.cursor and iteration


    csvFile1 = open(file, 'a' ,encoding='utf-8') #to be used at the very end
    count = 0
    count1 = 0
    numSuccessfullyAdded = 0
    # REINITIALIZE THE API every page or google blocks us. 
    translator = Translator()
    for page in tweepy.Cursor(api.search, q=keyword, lang='ar', geocode='17.8874,55.6121,300km',
                              count=200, include_rts=False, since=start_date).pages(50):

        #housekeeping
        count = count + 1
        print("beginning page ", count, " of 50 tweets each.")
        if(numSuccessfullyAdded >= numTweets):
          break



        #this says for tweet in a page of 50 tweets
        for status in page:

            #terminate if full
            if(numSuccessfullyAdded >= numTweets):
              print("number of desired tweets fullfilled. You should have ", numSuccessfullyAdded, " tweets in your file.")
              print("TERMINATE")
              break

            count1 = count1 + 1
            print("new tweet number: ", count1)
            print("Successfully added tweets fo far: ", numSuccessfullyAdded)

            #sleep for .5 seconds to avoid over requesting from google
            time.sleep(1.5)
            new_entry = []
            status = status._json
 
            ## check whether the tweet is in english or skip to the next tweet
            if status['lang'] != 'ar':
                continue
 
            #when run the code, below code replaces the retweet amount and
            #no of favorires that are changed since last download.
            if status['created_at'] in df['created_at'].values:
                i = df.loc[df['created_at'] == status['created_at']].index[0]
                if status['favorite_count'] != df.at[i, 'favorite_count'] or \
                   status['retweet_count'] != df.at[i, 'retweet_count']:
                    df.at[i, 'favorite_count'] = status['favorite_count']
                    df.at[i, 'retweet_count'] = status['retweet_count']
                continue
 
 
            #tweepy preprocessing called for basic preprocessing
            clean_text = p.clean(status['text'])

            #check the set and skip duplicate tweets even with different ids or authors
            if(clean_text in setOfTweets):
              print(clean_text)
              print("   tweet number ", count1, " has already been seen")
              continue
            setOfTweets.add(clean_text)
 
            #call clean_tweet method for extra preprocessing
            filtered_tweet=clean_tweets(clean_text)

            
            #translate the cleaned tweet using google trans
            print("   Try translating")            
            filtTweetCopy = copy.deepcopy(filtered_tweet)
            #print(filtTweetCopy)
            try:
              result = translator.translate(filtTweetCopy)
            except: 
              print("   TRANSLATION FAILED - If lots of these in a row, then google is blocking requests.")
              continue;
            print("   translating passed")
 
            #pass textBlob method for sentiment calculations
            blob = TextBlob(result.text)
            Sentiment = blob.sentiment
 
            #seperate polarity and subjectivity in to two variables
            polarity = Sentiment.polarity
            subjectivity = Sentiment.subjectivity

            print("   testing polar")
            if(polarity == 0 or subjectivity == 0):
              print("   polar failed")

              continue; 
            print("   polar passed")

            #new entry append
            new_entry += [status['id'], status['created_at'],
                          status['source'], status['text'],result.text, Sentiment,polarity,subjectivity, status['lang'],
                          status['favorite_count'], status['retweet_count']]
 
            #to append original author of the tweet
            new_entry.append(status['user']['screen_name'])
 
            try:
                is_sensitive = status['possibly_sensitive']
            except KeyError:
                is_sensitive = None
            new_entry.append(is_sensitive)
 
            # hashtagas and mentiones are saved using comma separted
            hashtags = ", ".join([hashtag_item['text'] for hashtag_item in status['entities']['hashtags']])
            new_entry.append(hashtags)
            mentions = ", ".join([mention['screen_name'] for mention in status['entities']['user_mentions']])
            new_entry.append(mentions)
 
            #get location of the tweet if possible
            try:
                location = status['user']['location']
            except TypeError:
                location = ''
            new_entry.append(location)
 
            try:
                coordinates = [coord for loc in status['place']['bounding_box']['coordinates'] for coord in loc]
            except TypeError:
                coordinates = None
            new_entry.append(coordinates)
 
            single_tweet_df = pd.DataFrame([new_entry], columns=COLS)
            #print(single_tweet_df)
            df = df.append(single_tweet_df, ignore_index=True)
            csvFile = open(file, 'a' ,encoding='utf-8')
            print("   SUCCESS with tweet: ", count1)
            numSuccessfullyAdded = numSuccessfullyAdded + 1
    df.to_csv(csvFile1, mode='a', columns=COLS, index=False, encoding="utf-8")

In [0]:
#OR #Iran OR #Iran OR #Rouhani OR #JCPOA
iran_sentiment_keywords = 'إيران OR روحاني OR خامنئي OR جهانگیری OR حاتمي OR ظریف OR واعظی OR #إيران OR #روحاني OR #خامنئي OR #جهانگیری OR #حاتمي OR #ظریف OR #واعظی'

In [0]:
#The tweets content will only be printed if it is a duplicate so if a 
#bunch of the same tweets are printed then that means duplicate detection
#is working. Everything else that is printed should be self explanatory. 
#Downloading can take a while now, so the printed "successfully added tweets so far"
#statement will tell you how far away you are from your max_number_of desired_tweets. 



#change this number to limit the number of tweets you want to pull. 
max_number_of_tweets_desired = 1000


#call main method passing keywords and file path
write_tweets(iran_sentiment_keywords,  kuwait_tweets, max_number_of_tweets_desired)

In [0]:
from google.colab import files
files.download('/content/sample_data/OMAN2_GREEN.csv')